- I used MMU2 dataset which contains 995 images corresponding to 100 people. 
- Each person was captured two eyes and each eye was captured 5 images (But one person is only captured one eye). 
- In which, with each eye, 3 random images are selected for the training set and the rest belong to the testing set. 

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import argparse, os
from cv2 import imread

from time import time
from scipy.io import savemat



In [3]:
from helpers.extractFeature import extractFeature
from helpers.segment import segment
from helpers.normalize import normalize
from helpers.encode import encode

In [4]:
# Segmentation parameters
eyelashes_thres = 80

# Normalisation parameters
radial_res = 20
angular_res = 240

# Feature encoding parameters
minWaveLength = 18
mult = 1
sigmaOnf = 0.5

In [5]:
# Description:
#     Extract features from an iris image

# Input:
#     im_filename - The input iris image
#     use_multiprocess - Use multiprocess to run

# Output:
#     template    - The extracted template
#     mask        - The extracted mask
#     im_filename - The input iris image
    
def extractFeature(im_filename, eyelashes_thres=80, use_multiprocess=True):

    # Perform segmentation
    im = imread(im_filename, 0)
    ciriris, cirpupil, imwithnoise = segment(im, eyelashes_thres, use_multiprocess)

    # Perform normalization
    polar_array, noise_array = normalize(imwithnoise, ciriris[1], ciriris[0], ciriris[2],
                                         cirpupil[1], cirpupil[0], cirpupil[2],
                                         radial_res, angular_res)

    # Perform feature encoding
    template, mask = encode(polar_array, noise_array, minWaveLength, mult, sigmaOnf)

    # Return
    return template, mask, im_filename

In [6]:

start = time()
file = "../input/CASIA1/1/001_1_1.jpg"

# Extract feature
print('>>> Enroll for the file ', file)


>>> Enroll for the file  ../input/CASIA1/1/001_1_1.jpg


In [7]:
template, mask, file = extractFeature(file)

In [9]:
# Save extracted feature
basename = os.path.basename(file)

out_file = os.path.join('./templates/', "%s.tmpl" % (basename))

savemat(out_file, mdict={'template':template, 'mask':mask})

print('>>> Template is saved in %s' % (out_file))

end = time()

print('>>> Enrollment time: {} [s]\n'.format(end-start))

>>> Template is saved in ./templates/001_1_1.jpg.tmpl
>>> Enrollment time: 27.629552125930786 [s]

